# t/&lambda; mapping example
In this example, it is shown on how to map the t/&lambda; from a low loss multispectrum. 

In [1]:
%matplotlib qt

In [2]:
import pyEELSMODEL.api as em
import os 
import numpy as np
import matplotlib.pyplot as plt

In [3]:
def make_circular_mask(xco, yco, Rin, Rout, shape):
    XX, YY = np.meshgrid(np.arange(shape[1])-yco, np.arange(shape[0])-xco)
    
    R = np.sqrt(XX**2+YY**2)
    
    mask = np.zeros(shape)
    boolean = (R>=Rin) & (R<Rout)
    mask[boolean] = 1
    return mask

def make_rectangular_mask(xco, yco, width,height, shape):
    mask = np.zeros(shape)
    mask[xco:xco+width, yco:yco+height] = 1
    return mask

### Simulation low loss
In the first part, a low loss with known t/&lambda; will be simulated. 

In [4]:
xsize = 128
ysize = 128
maps = np.zeros((2,xsize,ysize))

In [5]:
mask0 = make_rectangular_mask(5, 5, 20, 20, (xsize,ysize))
mask1 =  make_rectangular_mask(90, 90, 20, 30, (xsize,ysize))
mask2 = make_circular_mask(xsize//2, ysize//2, 20, 30, (xsize,ysize))
mask3 = make_circular_mask(xsize//2, ysize//2, 0, 20, (xsize,ysize))

In [6]:
cte=1
tlambda_map = np.ones_like(mask0)*0.3*cte
tlambda_map[mask0==1] = cte*0.5
tlambda_map[mask1==1] = cte*0.7
tlambda_map[mask2==1] = cte*1
tlambda_map[mask3==1] = cte*1.5

In [16]:
fig, ax = plt.subplots()
ax.imshow(tlambda_map)

The low loss will be modelled using the CoreLossSimulator, it is not needed to add the elements, edges and maps since we are not interested in the core-loss but will only calculate a low loss multispectrum

In [8]:
settings = (300e3, 1e-9, 20e-3)
msh = em.MultiSpectrumshape(0.5, 200, 2048, xsize, ysize) #will not be used
maps = np.zeros((2,xsize,ysize))
sh = msh.getspectrumshape()

sim = em.CoreLossSimulator(msh, [], [], maps, tlambda_map, settings)
sim.fwhm= 2
sim.n_plasmon = 3
sim.make_lowloss()

16384it [00:10, 1597.99it/s]


In [9]:
# add poisson noise to low loss
ll = sim.ll.copy()
cte = 1e3
ll.multidata = np.random.poisson(cte*ll.multidata)

In [10]:
em.MultiSpectrumVisualizer([ll])

No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.


### Extracting t/&lambda; from low loss
The t/&lambda; can be determined from the low loss by $t/\lambda = \ln \frac{I_t}{I_0}$ where $I_t$ is the total intensity and $I_0$ is the intensity of the zero loss peak. Two different methods can be used to extract $I_0$. 
1. Approximate the zero loss signal by taking the left part of the peak and doubling this.
2. Fit an appropriate function to the zero loss peak. A Gaussian or Lorentzian can be chosen.\
   
Both methods will be used to get a result which is confirmed by the ground truth value inputted

In [11]:
from pyEELSMODEL.operators.estimate_thickness import ThicknessEstimator

In [12]:
thick_mirror = ThicknessEstimator(sim.ll, model_type='Mirrored') #mirrored the zlp
thick_g = ThicknessEstimator(sim.ll, model_type='Gaussian') #fits gaussian to the zlp 

In [13]:
thick_mirror.log_ratio_method()
thick_g.log_ratio_method()

True


16384it [01:05, 251.96it/s]
16384it [00:01, 10954.32it/s]


In [17]:
fig, ax = plt.subplots(1,3)
ax[0].imshow(tlambda_map, vmin=0,vmax=1.5, cmap='jet')
ax[1].imshow(thick_mirror.tlambda,vmin=0,vmax=1.5,cmap='jet')
ax[1].set_title('Mirrored')
ax[2].imshow(thick_g.tlambda,vmin=0,vmax=1.5,cmap='jet')
ax[2].set_title('Gaussian model')

Text(0.5, 1.0, 'Gaussian model')